<a href="https://colab.research.google.com/github/MichaelCondo/Team9-APS360-Project/blob/main/CheckYoSelf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Check Yo' Self 
**Self-serve checkout enhancement using Neural Networks and Deep Learning**

## Architecture
![alt text](https://github.com/MichaelCondo/Team9-APS360-Project/blob/main/docs/Architecture_Diagram.png?raw=true)

#### Colab Link
https://colab.research.google.com/github/MichaelCondo/Team9-APS360-Project/blob/main/CheckYoSelf.ipynb

In [4]:
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

## 0. Helper Functions

## 1. Data Collection

**Tasks**:
* Understand Fruits 360 dataset
* Loading dataset into Colab
* Collecting prices of fruits
* Splitting the dataset into training, validation and testing (stratify or random) 

## 1.1 Fruits 360 Dataset

[Fruits 360](https://www.kaggle.com/moltean/fruits) is a dataset of 90000+ images of 130+ fruits and vegetables, in RGB colour at 100x100px a piece.

We're using [the GitHub mirror](https://github.com/Horea94/Fruit-Images-Dataset) to download the dataset as it does not require authentication to Kaggle.

In [152]:
# Download the dataset from GitHub
!wget -nc https://github.com/Horea94/Fruit-Images-Dataset/archive/master.zip \
    && mkdir -p "/root/project" \
    && unzip -n "master.zip" -d "/root/project/datasets" \
    && find "/root/project/datasets/" -mindepth 2 -maxdepth 2 -type d -ls
DATA_MASTER_PATH = "/root/project/datasets/Fruit-Images-Dataset-master"

Streaming output truncated to the last 5000 lines.
  inflating: /root/project/datasets/Fruit-Images-Dataset-master/Training/Hazelnut/239_100.jpg  
  inflating: /root/project/datasets/Fruit-Images-Dataset-master/Training/Hazelnut/23_100.jpg  
  inflating: /root/project/datasets/Fruit-Images-Dataset-master/Training/Hazelnut/240_100.jpg  
  inflating: /root/project/datasets/Fruit-Images-Dataset-master/Training/Hazelnut/241_100.jpg  
  inflating: /root/project/datasets/Fruit-Images-Dataset-master/Training/Hazelnut/242_100.jpg  
  inflating: /root/project/datasets/Fruit-Images-Dataset-master/Training/Hazelnut/243_100.jpg  
  inflating: /root/project/datasets/Fruit-Images-Dataset-master/Training/Hazelnut/244_100.jpg  
  inflating: /root/project/datasets/Fruit-Images-Dataset-master/Training/Hazelnut/245_100.jpg  
  inflating: /root/project/datasets/Fruit-Images-Dataset-master/Training/Hazelnut/246_100.jpg  
  inflating: /root/project/datasets/Fruit-Images-Dataset-master/Training/Hazelnut/247_

Let's take a look at the contents of the dataset. 

How many training samples are there for each of the classes?

In [153]:
!cd "{DATA_MASTER_PATH}/Training" \
    && find -maxdepth 2 -mindepth 2 -type f -printf "%h\0" \
    | uniq -zc | tr '\0' '\n' | sort -rh  \
    | (sed -u 10q ; echo "..." ; tail)

    984 ./Grape Blue
    900 ./Plum 3
    738 ./Tomato 3
    738 ./Tomato 1
    738 ./Strawberry Wedge
    738 ./Peach 2
    738 ./Melon Piel de Sapo
    738 ./Cherry Rainier
    738 ./Cherry 2
    735 ./Walnut
...
    438 ./Onion White
    429 ./Apple Red 3
    427 ./Avocado
    426 ./Mango Red
    420 ./Plum 2
    392 ./Cucumber Ripe
    367 ./Tomato Maroon
    300 ./Pear Kaiser
    300 ./Mangostan
    297 ./Ginger Root


### 1.2: Data Processing

Remove the folders that have items that are not sold individually, such as berries. Also, consolidate classes that feature the same type of fruit.

In [155]:
def remove_directories(dirs):
  for dir in dirs:
    !cd "{DATA_MASTER_PATH}/Training" \
    && echo "Removing Training: {dir}" && rm -r "{dir}"*

  for dir in dirs:
    !cd "{DATA_MASTER_PATH}/Test" \
    && echo "Removing Test: {dir}" && rm -r "{dir}"*

In [156]:
rm_dirs = ["Blueberry", "Cherry", "Chestnut", "Grape", "Hazelnut", "Mulberry", 
           "Nut", "Raspberry", "Redcurrant", "Strawberry", "Walnut"]
remove_directories(rm_dirs)

Removing Training: Blueberry
Removing Training: Cherry
Removing Training: Chestnut
Removing Training: Grape
Removing Training: Hazelnut
Removing Training: Mulberry
Removing Training: Nut
Removing Training: Raspberry
Removing Training: Redcurrant
Removing Training: Strawberry
Removing Training: Walnut
Removing Test: Blueberry
Removing Test: Cherry
Removing Test: Chestnut
Removing Test: Grape
Removing Test: Hazelnut
Removing Test: Mulberry
Removing Test: Nut
Removing Test: Raspberry
Removing Test: Redcurrant
Removing Test: Strawberry
Removing Test: Walnut


In [159]:
!cd "{DATA_MASTER_PATH}/Training" \
    && find -maxdepth 2 -mindepth 2 -type f -printf "%h\0" \
    | uniq -zc | tr '\0' '\n' | sort -rh  \
    | (sed -u 10q ; echo "..." ; tail)

    900 ./Plum 3
    738 ./Tomato 3
    738 ./Tomato 1
    738 ./Peach 2
    738 ./Melon Piel de Sapo
    711 ./Pear Stone
    702 ./Pepper Orange
    702 ./Pear Forelle
    702 ./Fig
    702 ./Cauliflower
...
    438 ./Onion White
    429 ./Apple Red 3
    427 ./Avocado
    426 ./Mango Red
    420 ./Plum 2
    392 ./Cucumber Ripe
    367 ./Tomato Maroon
    300 ./Pear Kaiser
    300 ./Mangostan
    297 ./Ginger Root


## 2. Data Processing

**Tasks**:
* Remove items that are not sold as individual fruits (i.e. vegetables, small berries, bunched bananas)
* Consolidate split classes either by combining or removing (e.g. Apple Red 1/2/3)


## 3. Baseline Model

**Tasks**:
* Setup and train random forrest model


## 4. Model Setup

**Tasks**:
* Using previous labs as reference, create CNN skeleton to work for 100x100 RGB images


## 5. Model Training and Tuning

**Tasks**:
* Write training code 
* Adjust hyperparameters to get at least 90% validation accuracy

